In [367]:
#importing necessary libs
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np

In [368]:
# loading json data as Pandas dataframe
json_data = pd.read_json('data/world_bank_projects.json')
json_data.columns

Index(['_id', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract', 'project_name', 'projectdocs',
       'projectfinancialtype', 'projectstatusdisplay', 'regionname', 'sector',
       'sector1', 'sector2', 'sector3', 'sector4', 'sector_namecode',
       'sectorcode', 'source', 'status', 'supplementprojectflg', 'theme1',
       'theme_namecode', 'themecode', 'totalamt', 'totalcommamt', 'url'],
      dtype='object')

In [369]:
country_projects = json_data[['countryname','project_name']]
country_projects.head()

,countryname,project_name
0,Federal Democratic Republic of Ethiopia,Ethiopia General Education Quality Improvement...
1,Republic of Tunisia,TN: DTF Social Protection Reforms Support
2,Tuvalu,Tuvalu Aviation Investment Project - Additiona...
3,Republic of Yemen,Gov't and Civil Society Organization Partnership
4,Kingdom of Lesotho,Second Private Sector Competitiveness and Econ...


In [370]:
countriesbyprojects= country_projects['countryname'].value_counts()
countriesbyprojects.head(10)

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64

In [371]:
#country name 'Africa' including borrowers from all aroung Africa continent and there's no country named Africa
#Looks like it should be dropped from the list
json_data[json_data['countryname']=='Africa']['borrower']

45                            ECOWAS
46                    UGANDA-COMOROS
51                  OSS, IUCN, CILSS
58                     BANK EXECUTED
65           BURUNDI,RWANDA,TANZANIA
99                GOVERNMENT OF MALI
167    MINISTRIES OF TELECOM AND ICT
184                         TANZANIA
287       MALAWI, MOZAMBIQUE, ZAMBIA
353            NILE BASIN INITIATIVE
449              GOVERNMENT OF NIGER
Name: borrower, dtype: object

In [372]:
#1. Find the 10 countries with most projects
countriesbyprojects = countriesbyprojects.drop('Africa')
toptencountries = countriesbyprojects.head(10)
toptencountries

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Republic of Mozambique             11
Islamic Republic of Pakistan        9
Name: countryname, dtype: int64

In [373]:
#Now starting #2 (Find the top 10 major project themes (using column 'mjtheme_namecode'))
json_data['mjtheme_namecode'].head()

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
Name: mjtheme_namecode, dtype: object

In [374]:
#needs to be normalized

data = json_data['mjtheme_namecode']

themes = pd.DataFrame(columns=['code','name'])
for row in data:
    themes = themes.append(json_normalize(row), ignore_index=True)
themes.head()

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


In [375]:
# 2. Top 10 major project themes

toptenthemes = themes['name'].value_counts().head(10)
toptenthemes

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

In [376]:
# 3. In 2. above you will notice that some entries have only the code and the name is missing. 
#Create a dataframe with the missing names filled in.

In [382]:
unique_namecode={}
for i in themes.code.unique():
#    print(themes.name[themes.code==i].iloc[7])
    if themes.name[themes.code==i].iloc[7] != '' :
        unique_namecode[i]=themes.name[themes.code==i].iloc[7]

themes.name=themes.code.map(lambda x: unique_namecode[x])
themes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 2 columns):
code    1499 non-null object
name    1499 non-null object
dtypes: object(2)
memory usage: 23.5+ KB


In [383]:
themes['name'].value_counts().head(10)

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64